In [1]:
from tabulate import tabulate

In [2]:
import wandb
import pandas as pd
import numpy as np

api = wandb.Api()

# Specify your project
project = "oualidzari/Link Inference In FL"

# Define your filters
filters = {
    "config.dataset": "Cora",
    "config.experiment_comment": "gradient defense"
}

# Fetch filtered runs
runs = api.runs(project, filters=filters)



In [22]:
data = []
for run in runs:
    name = run.name
    noise_level = run.config.get("gradient_defense_noise_level")

    # Fetch history
    history = run.history()
    accuracy_gradient = (
        history["Accuracy-gradients"].max()
        if "Accuracy-gradients" in history.columns
        else None
    )

    test_accuracy = (
        history["accuracy_test"].mean()
        if "accuracy_test" in history.columns
        else None
    )

    data.append(
        {
            "name": name,
            "noise_level": noise_level,
            "acc_grad": accuracy_gradient,
            "test_acc": test_accuracy,
        }
    )

In [23]:
df = pd.DataFrame(data)

In [24]:
df

,name,noise_level,acc_grad,test_acc
0,peach-valley-7169,0.00002,0.759046,0.710538
1,pretty-waterfall-7167,0.00003,0.706530,0.711418
2,apricot-energy-7169,0.00001,0.802534,0.720597
3,dandy-gorge-7170,0.00005,0.644899,0.699362
4,grateful-butterfly-7167,0.00004,0.684459,0.701521
5,elated-night-7163,0.00002,0.848870,0.724431
6,smart-grass-7162,0.00001,0.835391,0.759415
7,clear-energy-7163,0.00002,0.788558,0.726985
8,iconic-waterfall-7161,0.00004,0.736954,0.717921
9,eager-rain-7160,0.00005,0.669311,0.739715


In [25]:
# Group by budget and calculate statistics
grouped_stats = df.groupby('noise_level').agg({
    'acc_grad': ['mean', 'std'],

    'test_acc': ['mean', 'std']
}).sort_values(by='noise_level', ascending=True)

# Function to format mean and std as percentages
def format_mean_std(row):
    return f"{row['mean']*100:.2f} ± {row['std']*100:.2f}"

# Apply formatting
formatted_stats = pd.DataFrame()
for metric in ['acc_grad', 'test_acc']:
    formatted_stats[metric] = grouped_stats[metric].apply(format_mean_std, axis=1)

# Reset index to make budget a column and convert to percentage
formatted_stats = formatted_stats.reset_index()
# formatted_stats['budget'] = (formatted_stats['budget'] * 100).astype(int)

# Create a formatted table without row dividers
table = tabulate(formatted_stats, headers=['noise_level', 'acc_grad', 'test_acc'], 
                 tablefmt='simple', showindex=False)

print(table)

  noise_level  acc_grad      test_acc
-------------  ------------  ------------
       1e-05   81.68 ± 1.51  74.08 ± 1.86
       2e-05   79.88 ± 4.58  72.07 ± 0.89
       3e-05   74.15 ± 3.58  73.46 ± 2.03
       4e-05   70.60 ± 2.75  71.96 ± 1.90
       5e-05   66.03 ± 1.34  71.84 ± 2.03
       0.0001  59.54 ± 1.66  72.15 ± 2.21
       0.001   56.58 ± 0.91  71.18 ± 2.17
       0.01    55.80 ± 0.97  65.29 ± 2.28
       0.05    56.16 ± 0.70  50.11 ± 3.42
       0.1     56.59 ± 0.78  42.11 ± 3.24
       0.2     56.33 ± 0.60  41.19 ± 1.80
       0.4     56.30 ± 0.64  37.11 ± 1.99
       0.5     56.30 ± 0.65  38.32 ± 2.71
       1       56.53 ± 0.61  39.14 ± 1.28
       2       56.53 ± 0.60  35.77 ± 2.78
      10       56.53 ± 0.61  32.93 ± 1.48
